In [ ]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-Lib

In [ ]:
# Numpy のアップグレード
!pip install numpy --upgrade

# bokeh のバージョン指定
!pip install bokeh==2.4.3

ランタイムを再起動してください

In [1]:
# 環境変数を設定
!echo 'export LD_LIBRARY_PATH=/usr/lib' >> ~/.bashrc
!source ~/.bashrc

In [ ]:
!pip install backtesting

In [3]:
import pandas_datareader.data as pdr

def get_stock_data(code):
  df = pdr.DataReader("{}.JP".format(code), "stooq").sort_index()
  return df

In [4]:
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting.test import SMA

class SmaCross(Strategy):
    ns = 5 # 短期移動平均日数
    nl = 25 # 長期移動平均日数

    def init(self):
        # 短期移動平均
        self.smaS = self.I(SMA, self.data["Close"], self.ns)
        # 長期移動平均
        self.smaL = self.I(SMA, self.data["Close"], self.nl)

    def next(self):
        # smaS > smaL で買う
        if crossover(self.smaS, self.smaL):
            self.buy()
        # smaS < smaL で売る
        elif crossover(self.smaL, self.smaS):
            self.position.close()

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [5]:
from backtesting import Backtest
import datetime as dt

# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, SmaCross, trade_on_close=True)

# バックテストを実行
result = bt.run()
# 実行結果のデータを表示
print(result)
# 実行結果をグラフで表示
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   57.894737
Equity Final [$]                    16994.512
Equity Peak [$]                     24457.832
Return [%]                           69.94512
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                   55.207888
Volatility (Ann.) [%]               63.305004
Sharpe Ratio                         0.872094
Sortino Ratio                        2.220421
Calmar Ratio                          1.62263
Max. Drawdown [%]                  -34.023703
Avg. Drawdown [%]                   -6.130516
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       23 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                      65.268948
Worst Trade [%]                    -16.203019
Avg. Trade [%]                    

Row(id='1449', ...)

短期線と長期線の日数を最適化

In [6]:
result = bt.optimize(ns=range(5, 25, 5),nl=range(5, 75, 5), maximize="Return [%]", constraint=lambda r: r.ns < r.nl)
print(result)
bt.plot()

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   57.894737
Equity Final [$]                    22827.953
Equity Peak [$]                     23476.523
Return [%]                          128.27953
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                   98.221269
Volatility (Ann.) [%]               79.721742
Sharpe Ratio                         1.232051
Sortino Ratio                        4.185057
Calmar Ratio                         3.968653
Max. Drawdown [%]                  -24.749272
Avg. Drawdown [%]                   -6.105125
Max. Drawdown Duration      132 days 00:00:00
Avg. Drawdown Duration       24 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      74.132251
Worst Trade [%]                     11.321005
Avg. Trade [%]                    

Row(id='2181', ...)

RSIを使ったバックテスト

In [ ]:
import talib as ta

def RSI(close, n1, n2):
    rsiS = ta.RSI(close, timeperiod=n1)
    rsiL = ta.RSI(close, timeperiod=n2)
    return rsiS, rsiL

In [ ]:
class RSICross(Strategy):
    ns = 14 # 短期
    nl = 28 # 長期

    def init(self):
        self.rsiS, self.rsiL = self.I(RSI, self.data["Close"], self.ns, self.nl)

    def next(self):
        # 短期 >　長期で買い
        if crossover(self.rsiS, self.rsiL):
            self.buy()
        # 長期 > 短期で売り
        elif crossover(self.rsiL, self.rsiS):
            self.position.close()

In [ ]:
# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, RSICross, trade_on_close=True)

result = bt.run()
print(result)
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   46.052632
Equity Final [$]                    27044.385
Equity Peak [$]                     27216.245
Return [%]                          170.44385
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  128.123057
Volatility (Ann.) [%]               89.209222
Sharpe Ratio                         1.436209
Sortino Ratio                        7.965778
Calmar Ratio                        10.010991
Max. Drawdown [%]                  -12.798239
Avg. Drawdown [%]                   -3.927384
Max. Drawdown Duration       98 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   16
Win Rate [%]                            56.25
Best Trade [%]                       73.87128
Worst Trade [%]                     -3.977907
Avg. Trade [%]                    

Row(id='2956', ...)

RSI最適化

In [ ]:
result = bt.optimize(ns = range(5, 25, 5),
                     nl= range(5, 75, 5),
                      maximize = "Return [%]",
                      constraint = lambda r: r.ns < r.nl)

print(result)
bt.plot()

Backtest.optimize:   0%|          | 0/2 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   50.328947
Equity Final [$]                    30505.886
Equity Peak [$]                     30505.886
Return [%]                          205.05886
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  152.074286
Volatility (Ann.) [%]              101.482111
Sharpe Ratio                         1.498533
Sortino Ratio                        9.030565
Calmar Ratio                         8.573569
Max. Drawdown [%]                  -17.737571
Avg. Drawdown [%]                   -3.909969
Max. Drawdown Duration       94 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   23
Win Rate [%]                        47.826087
Best Trade [%]                       73.87128
Worst Trade [%]                     -4.215358
Avg. Trade [%]                    

Row(id='3786', ...)

MACDを使ったバックテスト

In [ ]:
def MACD(close, n1, n2, n3):
    macd, macdsignal, _ = ta.MACD(close, fastperiod=n1, slowperiod=n2, signalperiod=n3)
    return macd, macdsignal

In [ ]:
class MACDCross(Strategy):
    n1 = 12
    n2 = 26
    n3 = 9

    def init(self):
        self.macd, self.macdsignal = self.I(MACD, self.data["Close"], self.n1, self.n2, self.n3)

    def next(self):
        # macd >　シグナルで買い
        if crossover(self.macd, self.macdsignal):
            self.buy()
        # シグナル > macdで売り
        elif crossover(self.macdsignal, self.macd):
            self.position.close()

In [ ]:
# 新日本科学 （2395）の2021年1月〜2022年3月のデータ
df = get_stock_data(2395)
data = df[dt.datetime(2021,1,1):dt.datetime(2022,3,31)]

bt = Backtest(data, MACDCross, trade_on_close=True)

result = bt.run()
print(result)
bt.plot()

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   48.355263
Equity Final [$]                    23896.739
Equity Peak [$]                     28750.199
Return [%]                          138.96739
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  105.884106
Volatility (Ann.) [%]               81.673378
Sharpe Ratio                         1.296434
Sortino Ratio                        5.891296
Calmar Ratio                         5.271506
Max. Drawdown [%]                  -20.086122
Avg. Drawdown [%]                   -4.885147
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       22 days 00:00:00
# Trades                                   10
Win Rate [%]                             60.0
Best Trade [%]                      86.666769
Worst Trade [%]                     -8.612989
Avg. Trade [%]                    

Row(id='4615', ...)

In [ ]:
result = bt.optimize(n1=range(5, 75, 5), n2=range(10, 75, 5), n3=range(10, 75, 5), maximize = "Return [%]" , constraint = lambda r: r.n1 < r.n2)

print(result)
bt.plot()

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 1183 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/4 [00:00<?, ?it/s]

Start                     2021-01-04 00:00:00
End                       2022-03-31 00:00:00
Duration                    451 days 00:00:00
Exposure Time [%]                   47.697368
Equity Final [$]                    29876.953
Equity Peak [$]                     30210.503
Return [%]                          198.76953
Buy & Hold Return [%]              156.444719
Return (Ann.) [%]                  147.758623
Volatility (Ann.) [%]              100.074438
Sharpe Ratio                         1.476487
Sortino Ratio                        8.568002
Calmar Ratio                         9.154101
Max. Drawdown [%]                  -16.141249
Avg. Drawdown [%]                    -4.20448
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       21 days 00:00:00
# Trades                                   10
Win Rate [%]                             70.0
Best Trade [%]                      77.419673
Worst Trade [%]                    -10.353574
Avg. Trade [%]                    

Row(id='5444', ...)